<a href="https://colab.research.google.com/github/danm91/le_kingmakers/blob/testing/notebooks/Copy_of_roBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("mnt")

Mounted at mnt


In [2]:
%cd "mnt/My Drive/Colab Notebooks"

/content/mnt/My Drive/Colab Notebooks


In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import torch
!pip install transformers
import transformers

     |████████████████████████████████| 2.3MB 6.5MB/s 
     |████████████████████████████████| 901kB 26.6MB/s 
     |████████████████████████████████| 3.3MB 37.2MB/s 


In [5]:
from transformers import RobertaTokenizer

In [38]:
import pandas as pd
import re
import string
import pickle
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from kingmakers.preproc_text import process_tweets
from kingmakers.preproc_abbv import abbreviations, convert_abbrev_in_text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
df = pd.read_csv('kingmakers/training.1600000.processed.noemoticon.csv', encoding='latin', names = ['sentiment','id','date','query','user','tweet'])

In [39]:
df_test = df.sample(frac = 0.05, random_state=0)[['sentiment', 'tweet']]

In [40]:
df1 = df_test

In [41]:

df1['tweet'] = df1['tweet'].apply(lambda x: process_tweets(x))
df1['tweet'] = df1['tweet'].apply(lambda x: convert_abbrev_in_text(x))

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(df1['tweet'], df1['sentiment'], test_size=0.20,random_state=0)

In [42]:
X_train_token = [i.split() for i in df1['tweet']]

In [ ]:
X_train_token = [i.split() for i in lengths]


sentiment                                                    0
tweet        want compete want hard competition want rally ...
Name: 557138, dtype: object

,tweet
557138,want compete want hard competition want rally ...
349381,seems stuck ground amarillo put ground stop fl...
182051,pinking shear rarararrrarararrbabyproofing cut...
571236,0ff t0 meetin hate people v0lunteer free timegrrr
1339637,reply pls
...,...
1350300,think get raise mommy salary not cavity right ...
651945,heading dmv soon quotlegallyquot ride amp avoi...
390068,discouraged lack time management
1058149,good call sukses dong selalu buat kamu


In [43]:
maxlen = len(X_train_token[0])
for i in X_train_token:
  if len(i) > maxlen:
    maxlen = len(i) 

maxlen

32

In [47]:
target = target.map(lambda x: 1 if x==4 else 0)

In [45]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def roberta_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

557138     0
349381     0
182051     0
571236     0
1339637    1
          ..
1350300    1
651945     0
390068     0
1058149    1
956608     1
Name: sentiment, Length: 80000, dtype: int64

In [46]:
texts = df1['tweet']
target = df1['sentiment']

train_input_ids, train_attention_masks = roberta_encode(texts,32)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


557138     want compete want hard competition want rally ...
349381     seems stuck ground amarillo put ground stop fl...
182051     pinking shear rarararrrarararrbabyproofing cut...
571236     0ff t0 meetin hate people v0lunteer free timegrrr
1339637                                            reply pls
                                 ...                        
1350300    think get raise mommy salary not cavity right ...
651945     heading dmv soon quotlegallyquot ride amp avoi...
390068                      discouraged lack time management
1058149               good call sukses dong selalu buat kamu
956608                                  must watch go lakers
Name: tweet, Length: 80000, dtype: object

32

array([[  101,  2215,  5566, ...,     0,     0,     0],
       [  101,  3849,  5881, ...,     0,     0,     0],
       [  101,  5061,  2075, ...,     0,     0,     0],
       ...,
       [  101, 22585,  3768, ...,     0,     0,     0],
       [  101,  2204,  2655, ...,     0,     0,     0],
       [  101,  2442,  3422, ...,     0,     0,     0]])

In [48]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

def create_model(bert_model):
    
    input_ids = tf.keras.Input(shape=(32,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(32,),dtype='int32')

    output = roberta_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [44]:
from transformers import TFRobertaModel
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [49]:
model = create_model(roberta_model)
model.summary()           

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
tf_roberta_model_1 (TFRobertaMo TFBaseModelOutputWit 124645632   input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 32)           24608       tf_roberta_model_1[0][1]   

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(
    [train_input_ids, train_attention_masks],
    target,
    validation_split=0.2, 
    epochs=3,
    batch_size=10
)

Epoch 1/3
6400/6400 [==============================] - 429s 67ms/step - loss: 0.4132 - accuracy: 0.8142 - val_loss: 0.4218 - val_accuracy: 0.8111
Epoch 2/3
6400/6400 [==============================] - 429s 67ms/step - loss: 0.3979 - accuracy: 0.8212 - val_loss: 0.4176 - val_accuracy: 0.8090
Epoch 3/3
3628/6400 [================>.............] - ETA: 2:54 - loss: 0.3821 - accuracy: 0.8283

NameError: ignored

SyntaxError: ignored